In [1]:
# !pip install plotly
# !pip install seaborn
# !pip install matplotlib
# !pip install transformers
# !pip install tokenizers
# !pip install scikit-learn
# !pip install tensorflow
# !pip install pandas
# !pip install numpy
# !pip install keras_tuner


# from google.colab import drive
# drive.mount('/content/drive')

#
<a id="data-prep"></a>
<div style="background-color: #000D5B; color: white; text-align: center; padding: 6px 0 22px 0">
    <h3 style="background-color: #000D5B; color: white; text-align: left">RMIT School of Computer Science and Technology</h3>
    <br/>
    <h1>COSC3007: Deep Learning</h1>
    <h2>Assignment 2: Stance Twitter Sentiment Analysis and Detection </h2>
    <h2> [2] MODELLING AND MODEL EVALUATIONS </h2>

</div>

# Import Libs

In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import classification_report
import numpy as np

import numpy as np

# [1] Load Data

In [3]:
train_df = pd.read_csv("./drive/MyDrive/Colab Notebooks/StanceDataset/processed_train_augumented_vast.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)
train_ori = train_df.copy()

test_df = pd.read_csv("./drive/MyDrive/Colab Notebooks/StanceDataset/processed_test_augumented_vast.csv", encoding = "ISO-8859-1", engine="python").drop(columns=['Unnamed: 0'])

In [4]:
print(len(train_ori))
print(len(test_df))

16391
1956


# [2] Prepare label and fit data

## Split test and val

In [5]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Stance'])

## Tweets

In [6]:
from transformers import TFRobertaModel, RobertaTokenizer
import numpy as np

# Initialize the BERT tokenizer
MAX_LENGTH = 512
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = TFRobertaModel.from_pretrained('roberta-base')

# Assuming train_texts, val_texts, test_texts are your datasets
train_processed_tweets = train_df['Tweet'].tolist()
val_processed_tweets = val_df['Tweet'].tolist()
test_processed_tweets = test_df['Tweet'].tolist()

all_processed_tweets = train_processed_tweets + val_processed_tweets + test_processed_tweets
tokenized_tweets = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in all_processed_tweets]
MAX_LENGTH = max(len(tweet) for tweet in tokenized_tweets)
MAX_LENGTH = 128

print("Maximum length across all datasets:", MAX_LENGTH)

target_names = {
    0: "Hillary Clinton",
    1: "Legalization of Abortion",
    2: "Atheism",
    3: "Climate Change is a Real Concern",
    4: "Feminist Movement",
    5: "Donald Trump",
    6: "Joe Biden",
    7: "Barack Obama",
    8: "Kamala Harris",
    9: "Bernie Sanders",
    10: "Vu",
}

def bert_tokenize(texts, targets, max_length=MAX_LENGTH):
    input_ids = []
    attention_masks = []

    for idx, text in enumerate(texts):
        target_name = targets[idx]
        # target_name = target_names.get(targetID, "")
        formatted_text = f"<s> {target_name} </s></s> {text} </s>"
        encoded = tokenizer.encode_plus(
            formatted_text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='tf',
        )

        # Flatten the tensors and remove the extra dimension
        input_ids.append(tf.squeeze(encoded['input_ids'], axis=0))
        attention_masks.append(tf.squeeze(encoded['attention_mask'], axis=0))

    return np.array(input_ids), np.array(attention_masks)

# Assuming you have lists of preprocessed texts for training, validation, and test sets
train_input_ids, train_attention_masks = bert_tokenize(train_df['Tweet'],  train_df['Target'].to_list())
val_input_ids, val_attention_masks = bert_tokenize(val_df['Tweet'], val_df['Target'].to_list())
test_input_ids, test_attention_masks = bert_tokenize(test_df['Tweet'], test_df['Target'].to_list())

print(train_input_ids.shape)
print(val_input_ids.shape)
print(test_input_ids.shape)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Maximum length across all datasets: 128
(13112, 128)
(3279, 128)
(1956, 128)


## Stances

In [7]:
def categorized_label(df, label_name):
    label_encoder = LabelEncoder()
    encoded_labels = label_encoder.fit_transform(df[label_name])
    categorical_labels = to_categorical(encoded_labels)
    return categorical_labels

In [8]:
train_labels = categorized_label(train_df, "Stance")
test_labels = categorized_label(test_df, "Stance")
val_labels = categorized_label(val_df, "Stance")

# [3] Create Model

In [9]:
from keras import backend as K
from tensorflow.keras.optimizers.legacy import Adam
from hyperopt import hp
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
# import keras_tuner as kt

In [10]:
def f1_score_class(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

def f1_score(y_true, y_pred):
    # Convert predictions to one-hot format
    y_pred_one_hot = K.one_hot(K.argmax(y_pred), num_classes=3)

    # Calculate F1 score for each class
    f1s = [f1_score_class(y_true[:, i], y_pred_one_hot[:, i]) for i in range(3)]

    # Average F1 scores across all classes
    return K.mean(K.stack(f1s), axis=0)

Set up call backs and learning rates

In [11]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


# Define the EarlyStopping and ReduceLROnPlateau callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    verbose=1,
    restore_best_weights=True,
)

early_stopping2 = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True,
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    min_lr=1e-5,
    verbose=1,
)

# Define the ModelCheckpoint callback
checkpoint_path = "roberta_best_model.h5"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,    # Only save a model if 'val_loss' has improved
    monitor='val_loss',     # Monitor 'val_loss' during training
    mode='min',             # The model is saved when 'val_loss' is minimized
    verbose=1)

# Combine all callbacks in a list
callbacks = [
    early_stopping,
    lr_scheduler,
    model_checkpoint_callback
]

# Combine all callbacks in a list
callbacks2 = [
    early_stopping2,
    lr_scheduler,
    model_checkpoint_callback
]

In [12]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization, Bidirectional, Flatten, Input, GlobalMaxPooling1D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers.legacy import Adam
# import keras_tuner as kt
from transformers import TFRobertaModel

def build_model(drop, regr, lr):
    bert_model = TFRobertaModel.from_pretrained('roberta-base')
    bert_model.trainable = False

    # Define model input (BERT expects two inputs: input_ids and attention_mask)
    input_ids = Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='input_ids')
    attention_mask = Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='attention_mask')

    # Get BERT embeddings
    bert_output = bert_model(input_ids, attention_mask=attention_mask)[0]  # Use the pooled output here

    # Tuning the number of units in the first biLSTM layer
    x = Bidirectional(LSTM(units=64,  dropout=drop, recurrent_dropout=drop,return_sequences=True,kernel_regularizer=tf.keras.regularizers.l2(regr))) (bert_output)

    x = GlobalMaxPooling1D()(x)

    # Dense layer with tunable units
    x = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(regr)) (x)

    # Tunable dropout rate for the second layer
    x = Dropout(drop)(x)

    # Final Layer
    x = Dense(3, activation='softmax')(x)

    # Tuning the learning rate for the optimizer
    optimizer = Adam(learning_rate=lr)
    model = Model(inputs=[input_ids, attention_mask], outputs=x)
    model.compile(loss='CategoricalCrossentropy', optimizer=optimizer, metrics=['categorical_accuracy',f1_score])
    model.summary()

    return model



Recreate the best model with more training time

In [13]:
history = {}
inital_model = build_model(0.8, 0.0005, 0.001)
history['tuned_stance_model_custom'] = inital_model.fit([train_input_ids, train_attention_masks], train_labels,
                    epochs=1000,
                    validation_data=([val_input_ids, val_attention_masks], val_labels),
                    batch_size=256,
                    callbacks=callbacks)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.dense.weight']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_roberta_model_1 (TFRobe  TFBaseModelOutputWithPooli   1246456   ['input_ids[0][0]',           
 rtaModel)                   ngAndCrossAttentions(last_   32         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

In [14]:
results = inital_model.evaluate([test_input_ids, test_attention_masks], test_labels)
print('Test accuracy:', results)

62/62 [==============================] - 9s 138ms/step - loss: 0.8456 - categorical_accuracy: 0.6401 - f1_score: 0.4413
Test accuracy: [0.8456452488899231, 0.6400818228721619, 0.4413057565689087]


In [15]:
# Get model predictions
predictions = inital_model.predict([test_input_ids, test_attention_masks])

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Assuming test_labels are one-hot encoded, convert them to class labels
true_labels = np.argmax(test_labels, axis=1)

# Generate classification report
report = classification_report(true_labels, predicted_labels)
print(report)

62/62 [==============================] - 12s 136ms/step
              precision    recall  f1-score   support

           0       0.66      0.77      0.71      1014
           1       0.64      0.40      0.49       452
           2       0.59      0.59      0.59       490

    accuracy                           0.64      1956
   macro avg       0.63      0.59      0.60      1956
weighted avg       0.64      0.64      0.63      1956



In [16]:
def load_classification_rp_per_target(model):
    targets = test_df.Target.unique()
    metrics_df = pd.DataFrame()
    for target in targets:
        extracted_df = test_df[test_df['Target'] == target]
        input_ids, attention_masks = bert_tokenize(extracted_df['Tweet'], extracted_df['Target'].to_list())
        labels = categorized_label(extracted_df, "Stance")

        # Convert one-hot encoded labels to single integer class labels
        true_labels = np.argmax(labels, axis=1)

        # Get model predictions
        predictions = model.predict([input_ids, attention_masks])

        # Convert predictions to class labels
        predicted_labels = np.argmax(predictions, axis=1)

        # Generate a classification report
        report = classification_report(true_labels, predicted_labels, target_names=['AGAINST', 'FAVOR', 'NONE'])
        target_name = target
        # if target == 0:
        #     target_name = "Hillary Clinton"
        # if target == 1:
        #     target_name = "Legalization of Abortion"
        # if target == 2:
        #     target_name = "Atheism"
        # if target == 3:
        #     target_name = "Climate Change is a Real Concern"
        # if target == 4:
        #     target_name = "Feminist Movement"
        # if target == 5:
        #     target_name = "Donald Trump"

        print('Target: ', target_name)
        print(report)
        print("-----------------------------------------", "\n")

        # Flatten the report into a single row and add target name
        report = classification_report(true_labels, predicted_labels, target_names=['AGAINST', 'FAVOR', 'NONE'], output_dict=True)

        # Extract weighted f1, macro f1, and accuracy
        report_data = {
            'Target': target_name,
            'Weighted F1': report['weighted avg']['f1-score'],
            'Macro F1': report['macro avg']['f1-score'],
            'Accuracy': report['accuracy']
        }

        # Append to the DataFrame
        metrics_df = metrics_df.append(report_data, ignore_index=True)
    metrics_df.set_index('Target', inplace=True)

    return metrics_df

In [17]:
# Use the function to get the metrics DataFrame
model_metrics_df = load_classification_rp_per_target(inital_model)
model_metrics_df

7/7 [==============================] - 1s 134ms/step
Target:  Atheism
              precision    recall  f1-score   support

     AGAINST       0.75      0.90      0.82       160
       FAVOR       0.50      0.03      0.06        32
        NONE       0.35      0.32      0.33        28

    accuracy                           0.70       220
   macro avg       0.53      0.42      0.40       220
weighted avg       0.66      0.70      0.65       220

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


6/6 [==============================] - 1s 130ms/step
Target:  Climate Change is a Real Concern
              precision    recall  f1-score   support

     AGAINST       0.33      0.09      0.14        11
       FAVOR       0.79      0.91      0.85       123
        NONE       0.50      0.34      0.41        35

    accuracy                           0.74       169
   macro avg       0.54      0.45      0.46       169
weighted avg       0.70      0.74      0.71       169

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


9/9 [==============================] - 1s 136ms/step
Target:  Feminist Movement
              precision    recall  f1-score   support

     AGAINST       0.67      0.74      0.71       183
       FAVOR       0.30      0.33      0.31        58
        NONE       0.58      0.25      0.35        44

    accuracy                           0.58       285
   macro avg       0.52      0.44      0.46       285
weighted avg       0.58      0.58      0.57       285

----------------------------------------- 



...

9/9 [==============================] - 1s 134ms/step
Target:  Legalization of Abortion
              precision    recall  f1-score   support

     AGAINST       0.73      0.91      0.81       189
       FAVOR       0.50      0.09      0.15        46
        NONE       0.62      0.51      0.56        45

    accuracy                           0.71       280
   macro avg       0.62      0.50      0.51       280
weighted avg       0.68      0.71      0.66       280

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


23/23 [==============================] - 3s 135ms/step
Target:  Donald Trump
              precision    recall  f1-score   support

     AGAINST       0.52      0.63      0.57       299
       FAVOR       0.75      0.18      0.29       148
        NONE       0.61      0.73      0.67       260

    accuracy                           0.57       707
   macro avg       0.63      0.51      0.51       707
weighted avg       0.60      0.57      0.55       707

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


,Weighted F1,Macro F1,Accuracy
Target,,,
Atheism,0.646022,0.403446,0.700000
Climate Change is a Real Concern,0.708749,0.464973,0.739645
Feminist Movement,0.570944,0.455725,0.582456
Hillary Clinton,0.676516,0.607158,0.688136
Legalization of Abortion,0.662137,0.506815,0.710714
Donald Trump,0.547472,0.509846,0.572843


[4] Fine Tune

In [19]:
for layer in inital_model.layers:
    layer.trainable = True
   # Tuning the learning rate for the optimizer
inital_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(1e-7),
                  loss='categorical_crossentropy',
                  metrics=['categorical_accuracy', f1_score])

history['fine_tuned_bert'] = inital_model.fit([train_input_ids, train_attention_masks], train_labels,
                    epochs=1000,
                    validation_data=([val_input_ids, val_attention_masks], val_labels),
                    batch_size=32,
                    callbacks=callbacks2)

Epoch 1/1000


410/410 [==============================] - ETA: 0s - loss: 0.8733 - categorical_accuracy: 0.6245 - f1_score: 0.5956
Epoch 1: saving model to roberta_best_model.h5
410/410 [==============================] - 362s 831ms/step - loss: 0.8733 - categorical_accuracy: 0.6245 - f1_score: 0.5956 - val_loss: 0.8373 - val_categorical_accuracy: 0.6340 - val_f1_score: 0.6193 - lr: 1.0000e-07
Epoch 2/1000
410/410 [==============================] - ETA: 0s - loss: 0.8688 - categorical_accuracy: 0.6317 - f1_score: 0.6045
Epoch 2: saving model to roberta_best_model.h5
410/410 [==============================] - 337s 823ms/step - loss: 0.8688 - categorical_accuracy: 0.6317 - f1_score: 0.6045 - val_loss: 0.8221 - val_categorical_accuracy: 0.6404 - val_f1_score: 0.6257 - lr: 1.0000e-07
Epoch 3/1000
410/410 [==============================] - ETA: 0s - loss: 0.8593 - categorical_accuracy: 0.6306 - f1_score: 0.6051
Epoch 3: saving model to roberta_best_model.h5
410/410 [==============================] - 339s 8

In [20]:
# Use the function to get the metrics DataFrame
model_metrics_df = load_classification_rp_per_target(inital_model)
model_metrics_df

7/7 [==============================] - 4s 135ms/step
Target:  Atheism
              precision    recall  f1-score   support

     AGAINST       0.76      0.86      0.81       160
       FAVOR       0.33      0.06      0.11        32
        NONE       0.39      0.46      0.43        28

    accuracy                           0.70       220
   macro avg       0.50      0.46      0.45       220
weighted avg       0.65      0.70      0.66       220

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


6/6 [==============================] - 1s 131ms/step
Target:  Climate Change is a Real Concern
              precision    recall  f1-score   support

     AGAINST       0.00      0.00      0.00        11
       FAVOR       0.84      0.89      0.87       123
        NONE       0.59      0.66      0.62        35

    accuracy                           0.78       169
   macro avg       0.48      0.51      0.50       169
weighted avg       0.74      0.78      0.76       169

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


9/9 [==============================] - 1s 135ms/step
Target:  Feminist Movement
              precision    recall  f1-score   support

     AGAINST       0.72      0.61      0.66       183
       FAVOR       0.34      0.57      0.43        58
        NONE       0.41      0.32      0.36        44

    accuracy                           0.55       285
   macro avg       0.49      0.50      0.48       285
weighted avg       0.59      0.55      0.56       285

----------------------------------------- 



...

9/9 [==============================] - 1s 134ms/step
Target:  Legalization of Abortion
              precision    recall  f1-score   support

     AGAINST       0.75      0.87      0.81       189
       FAVOR       0.75      0.13      0.22        46
        NONE       0.52      0.60      0.56        45

    accuracy                           0.71       280
   macro avg       0.67      0.53      0.53       280
weighted avg       0.71      0.71      0.67       280

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


23/23 [==============================] - 3s 135ms/step
Target:  Donald Trump
              precision    recall  f1-score   support

     AGAINST       0.57      0.63      0.60       299
       FAVOR       0.82      0.28      0.41       148
        NONE       0.67      0.85      0.75       260

    accuracy                           0.64       707
   macro avg       0.69      0.59      0.59       707
weighted avg       0.66      0.64      0.62       707

----------------------------------------- 



<ipython-input-16-653adaea9b0f>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(report_data, ignore_index=True)


,Weighted F1,Macro F1,Accuracy
Target,,,
Atheism,0.658201,0.446959,0.695455
Climate Change is a Real Concern,0.758352,0.495567,0.781065
Feminist Movement,0.564376,0.481450,0.554386
Hillary Clinton,0.743906,0.708286,0.745763
Legalization of Abortion,0.670599,0.528590,0.707143
Donald Trump,0.616110,0.588043,0.636492


In [21]:
# Get model predictions
predictions = inital_model.predict([test_input_ids, test_attention_masks])

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Assuming test_labels are one-hot encoded, convert them to class labels
true_labels = np.argmax(test_labels, axis=1)

# Generate classification report
report = classification_report(true_labels, predicted_labels)
print(report)

62/62 [==============================] - 8s 136ms/step
              precision    recall  f1-score   support

           0       0.70      0.72      0.71      1014
           1       0.66      0.48      0.55       452
           2       0.62      0.75      0.68       490

    accuracy                           0.67      1956
   macro avg       0.66      0.65      0.65      1956
weighted avg       0.67      0.67      0.67      1956



In [22]:
inital_model.save('roberta_model_vast_66.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
